In [ ]:
%%bigquery --project projchallenge

#initial cohorts creation based on year and month
SELECT
  FORMAT_DATE('%Y-%m', subscription_started) AS cohort,
  subscription_plan_type,
  COUNT(*) AS total_subscriptions
FROM
  `projchallenge.subscription_sorted.round2_technical_interview_sorted_data`
GROUP BY
  FORMAT_DATE('%Y-%m',subscription_started),
  subscription_plan_type
ORDER BY
  cohort ASC,
  subscription_plan_type ASC;

In [ ]:
%%bigquery --project projchallenge
#dynamic retention periods
WITH cohorts AS (
  SELECT
    FORMAT_DATE('%Y-%m', DATE(subscription_started)) AS subscription_started,
    DATE_TRUNC(DATE(subscription_started), MONTH) AS cohort_start_date,
    CASE
      WHEN cancel_time is NULL THEN LAST_DAY(CURRENT_DATE())
      WHEN cancel_time = '' THEN LAST_DAY(CURRENT_DATE())
      WHEN DATE(cancel_time) = DATE(subscription_started) THEN DATE_ADD(DATE(subscription_started) , INTERVAL 1 DAY)
      ELSE DATE(cancel_time)
    END AS subscription_cancel_time,
    subscription_plan_type,

  FROM
    `projchallenge.subscription_sorted.round2_technical_interview_sorted_data`
),
subscription_periods AS(
  SELECT
    subscription_started, subscription_plan_type,
    DATE_ADD(cohort_start_date, INTERVAL month_offset MONTH) AS active_month,
    month_offset +1 AS month_number,
    subscription_cancel_time
  FROM
    cohorts
  CROSS JOIN UNNEST(
    GENERATE_ARRAY(0, DATE_DIFF(
      subscription_cancel_time,
      cohort_start_date,
      MONTH)
    )
  ) AS month_offset
  WHERE DATE_ADD(cohort_start_date, INTERVAL month_offset MONTH) <= subscription_cancel_time
),
retention_table AS(
  SELECT
    subscription_started,
    month_number,
    subscription_plan_type,
    COUNT(*) AS active_subscriptions
  FROM
    subscription_periods
  GROUP BY
    subscription_started,
    month_number,
    subscription_plan_type
 )

SELECT
  subscription_started, subscription_plan_type,
  MAX(CASE WHEN month_number = 1 THEN active_subscriptions ELSE 0 END) AS month_1,
  MAX(CASE WHEN month_number = 2 THEN active_subscriptions ELSE 0 END) AS month_2,
  MAX(CASE WHEN month_number = 3 THEN active_subscriptions ELSE 0 END) AS month_3,
  MAX(CASE WHEN month_number = 4 THEN active_subscriptions ELSE 0 END) AS month_4,
  MAX(CASE WHEN month_number = 5 THEN active_subscriptions ELSE 0 END) AS month_5,
  MAX(CASE WHEN month_number = 6 THEN active_subscriptions ELSE 0 END) AS month_6,
  MAX(CASE WHEN month_number = 7 THEN active_subscriptions ELSE 0 END) AS month_7,
  MAX(CASE WHEN month_number = 8 THEN active_subscriptions ELSE 0 END) AS month_8,
  MAX(CASE WHEN month_number = 9 THEN active_subscriptions ELSE 0 END) AS month_9,
  MAX(CASE WHEN month_number = 10 THEN active_subscriptions ELSE 0 END) AS month_10,
  MAX(CASE WHEN month_number = 11 THEN active_subscriptions ELSE 0 END) AS month_11,
  MAX(CASE WHEN month_number = 12 THEN active_subscriptions ELSE 0 END) AS month_12
FROM
  retention_table
GROUP BY
    subscription_started, subscription_plan_type
ORDER BY
   subscription_started, subscription_plan_type;

